# Group 2 Lego Dataset Report

### Libraries Loading

In [18]:
import pandas as pd
import re

In [19]:
pd.options.display.max_rows = 4000


In [4]:
print("Hi")

Hi


In [5]:
!pip install openpyxl

In [5]:
import os
os.getcwd()

'/Users/Anna/Desktop/DATAANALYTICS/asda2025/Portfolio/ASDA_2025_Group_2_Portfolio_/notebooks'

## Data Loading

In [20]:
dataset = pd.read_excel("../additional_materials/Lego_Database1-11-2025.xlsx", sheet_name=None)

## Data Initial Inspection

In [21]:
dataset = pd.concat(dataset.values(), ignore_index=True)


In [22]:
dataset["id"] = [i for i in range(len(dataset))]

In [10]:
dataset.dtypes

id                   int64
color               object
is duplo?           object
size type           object
base shape          object
base dimensions     object
number of studs    float64
has slope?          object
slope degree       float64
in stock            object
transparent        float64
dtype: object

In [12]:
dataset.shape

(204, 11)

In [24]:
dataset.isnull().sum()

id                   0
color                0
is duplo?            0
size type            0
base shape           0
base dimensions      0
number of studs      0
has slope?           0
slope degree        36
in stock             0
transparent        164
dtype: int64

In [23]:
dataset["transparent"].value_counts()

transparent
0.0    36
1.0     4
Name: count, dtype: int64

In [25]:
dataset["base dimensions"].unique()

array(['2 x 4', '2 x 2', '2 x 8', '1 x 4', '1 x 3', '2 x 6', '1 x 2',
       '4 x 6', '1 x 1', '2 x 3', '0 x 0 ', '0 x 0', '2 x 4 + 2 x 2',
       '2*4', '2*8', '2*2', '2*6', '2*3', '1*3', '1*4', '1*2', '1*1',
       '4*6', '4 X 2', '2 X 2', '8 X 2', '6 X 2', '4 X 1', '2 X 1',
       '3 X 2', '6 X 4', '1 X 1', '4 X 4', '3 X 1', '2x4', '2x8', '2x2',
       '2x6', '2x3', '1x2', '1x4', '1x3', '1x1', '4x4', '4x6', '4*4'],
      dtype=object)

## Data Cleaning

In [26]:
dataset = dataset.drop(dataset[dataset["base dimensions"] == "2 x 4 + 2 x 2"].index)


In [27]:
def clean_dimensions(val):
    if pd.isna(val):
        return val
    val = str(val).strip()
    val = re.sub(r'\s*[xX×∗+]\s*', '*', val)
    val = val.strip()
    return val

dataset["base dimensions"] = dataset["base dimensions"].apply(clean_dimensions)

In [28]:
dataset["base dimensions"].value_counts()

base dimensions
2*2    47
1*1    33
1*2    29
2*4    21
1*4    12
2*8     8
2*3     7
4*2     7
2*1     7
1*3     5
0*0     5
2*6     5
4*6     4
4*1     3
4*4     3
8*2     2
3*2     2
6*2     1
6*4     1
3*1     1
Name: count, dtype: int64

In [81]:
for i in dataset.columns:
    if dataset[i].dtype == 'object':
        print(dataset[i].value_counts())
        print("*"*20)

color
yellow                  9
white                   9
grey                    9
red                     8
black                   8
blue                    8
Yellow                  7
Black                   6
orange                  6
Red                     5
Purple                  5
White                   4
pink                    4
Beige                   4
green                   4
darkblue                4
Dark Blue               4
lightpink               3
Green                   3
Pink                    3
Light Green             3
navy blue               3
Grey                    3
turquoise               3
purple                  2
pear                    2
neon yellow             2
Green                   2
brick red               2
lightbrown              2
dark green              2
Orange                  2
Brown                   2
Sky Blue                2
Transparent yellow      2
Blue                    2
bright green            2
lightgreen              2
darkpi

In [82]:
dataset["transparent"]=dataset["transparent"].fillna(0)

In [83]:
dataset = dataset.map(lambda x: x.lower() if isinstance(x, str) else x)


In [84]:
dataset["is duplo?"].value_counts()

is duplo?
False    102
no        68
True      19
yes       14
Name: count, dtype: int64

In [86]:
dataset = dataset.map(
    lambda x: True if isinstance(x, str) and x.strip().lower() == "yes"
    else False if isinstance(x, str) and x.strip().lower() == "no"
    else x
)


In [87]:
# Step 1: Lowercase and strip spaces
dataset["base shape"] = dataset["base shape"].str.strip()

# Step 2: Replace similar or synonymous shapes
dataset["base shape"] = dataset["base shape"].replace({
    "trapezium": "trapezoid",  # same shape
    "round": "circle",         # unify round → circle
    "wadge": "triangle"        # close in shape sense, optional
})

# Step 3: Check clean counts
print(dataset["base shape"].value_counts())


base shape
rectangle    109
square        59
circle        24
trapezoid      8
triangle       3
Name: count, dtype: int64


In [88]:
dataset["color"].value_counts()

color
yellow                  16
black                   14
red                     13
white                   13
grey                    12
blue                    10
orange                   8
green                    7
pink                     7
purple                   7
sky blue                 5
dark blue                4
beige                    4
darkblue                 4
navy blue                3
dark green               3
turquoise                3
lightpink                3
light green              3
neon yellow              3
coral                    2
darkpink                 2
bright green             2
lightbrown               2
cyan                     2
brick red                2
transparent yellow       2
green                    2
pear                     2
bright yellow            2
brown                    2
lightgreen               2
cream                    2
lilac                    1
red brown                1
orchid                   1
darkpurple            

In [89]:
dataset["size type"] = dataset["size type"].replace({
    "tile": "plate",  
})

In [90]:
dataset["size type"].value_counts()

size type
plate    107
brick     96
Name: count, dtype: int64

In [91]:
dataset["slope degree"]=dataset["slope degree"].fillna(0)

In [92]:
dataset["in stock"]= dataset["in stock"].replace(True, 1)


/var/folders/lc/rwbnk4114w78tcfk73z_364m0000gn/T/ipykernel_22103/2050416269.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  dataset["in stock"]= dataset["in stock"].replace(True, 1)


In [93]:
dataset["in stock"].astype(str).value_counts()

in stock
1    203
Name: count, dtype: int64

In [94]:
dataset["has slope?"].unique()

array([False, True], dtype=object)

In [98]:
dataset["has slope?"] = dataset["has slope?"].replace({0: False, 1: True})

/var/folders/lc/rwbnk4114w78tcfk73z_364m0000gn/T/ipykernel_22103/3524988637.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  dataset["has slope?"] = dataset["has slope?"].replace({0: False, 1: True})


## AI was used for the following code

In [99]:
import re
import pandas as pd

def extract_base_color(df, color_col='color'):
    # set of base colors you want to end up with
    BASES = {
        'blue','green','yellow','pink','black','grey','gray','white',
        'purple','orange','beige','brown','red','cream', 'transparent'
    }

    # words to drop wherever they appear (prefix/postfix/inline)
    DESCRIPTORS = {
        'dark','light','neon','bright','deep','shiny','transparent','baby'
    }

    # synonyms or rare shades -> base
    SYN_MAP = {
        # blue family
        'navy':'blue','aqua':'blue','turquoise':'blue','cyan':'blue','teal':'blue',
        'capri':'blue','sky':'blue','denim':'blue','ice':'blue',
        # green family
        'lime':'green','olive':'green','pear':'green',
        # pink/purple family
        'magenta':'pink','coral':'pink','peach':'pink','violet':'purple',
        'lilac':'purple','orchid':'purple',
        # red family
        'maroon':'red',
        # beige/brown family
        'khaki':'beige','cafe':'brown','mud':'brown',
        # transparent family
        'Transparent':'transparent',
    }

    # quick typo fixes
    TYPO = {'blck':'black','gry':'grey','magent':'magenta','lite':'light'}

    # prebuild a regex to split non-letters to spaces
    nonletter = re.compile(r'[_\-]+')

    # for concatenations like "darkblue", "mudbrown", try to separate descriptors/synonyms from bases
    attach_prefixes = tuple(list(DESCRIPTORS) + list(SYN_MAP.keys()))

    def clean_one(x):
        if pd.isna(x): 
            return None
        s = str(x).strip().lower()
        # typos
        for bad, good in TYPO.items():
            s = s.replace(bad, good)
        # normalize separators
        s = nonletter.sub(' ', s)
        s = re.sub(r'\s+', ' ', s).strip()

        # insert spaces in concatenations like "darkblue", "mudbrown"
        for p in attach_prefixes:
            s = re.sub(fr'(?<={p})(?=[a-z])', ' ', s)

        # tokenise and drop descriptors
        tokens = [t for t in re.split(r'\s+', s) if t]

        tokens = [t for t in tokens if t not in DESCRIPTORS]

        # map synonyms token-wise
        tokens = [SYN_MAP.get(t, t) for t in tokens]

        # if any token is exactly a base, pick the LAST one (e.g., "red brown" -> "brown")
        bases_in_tokens = [t for t in tokens if t in BASES]
        if bases_in_tokens:
            base = bases_in_tokens[-1]
        else:
            # no exact base token: try suffix match (handles "mudbrown", "redbrown")
            base = None
            for b in BASES:
                if s.endswith(b) or ''.join(tokens).endswith(b):
                    base = b
                    break

        return base

    df['Base color'] = df[color_col].apply(clean_one)
    return df


In [100]:
dataset = extract_base_color(dataset, 'color')

In [101]:
dataset

,id,color,is duplo?,size type,base shape,base dimensions,number of studs,has slope?,slope degree,in stock,transparent,Base color
0,0,dark blue,True,brick,rectangle,2*4,8,False,0.0,1,0.0,blue
1,1,green,True,brick,rectangle,2*4,8,False,0.0,1,0.0,green
2,2,coral,True,brick,square,2*2,4,False,0.0,1,0.0,pink
3,3,red,True,brick,square,2*2,4,False,0.0,1,0.0,red
4,4,white,True,brick,square,2*2,4,False,0.0,1,0.0,white
5,5,green,True,plate,rectangle,2*4,8,False,0.0,1,0.0,green
6,6,light green,True,plate,rectangle,2*8,16,False,0.0,1,0.0,green
7,7,orange,False,brick,square,2*2,4,False,0.0,1,0.0,orange
8,8,orange,False,plate,rectangle,1*4,4,False,0.0,1,0.0,orange
9,9,yellow,False,brick,rectangle,2*4,8,False,0.0,1,0.0,yellow


In [40]:
dataset.columns

Index(['id', 'color', 'is duplo?', 'size type', 'base shape',
       'base dimensions', 'number of studs', 'has slope?', 'slope degree',
       'in stock', 'transparent', 'Base color'],
      dtype='object')

In [ ]:
dataset[dataset["Base color"].isna()]

,id,color,is duplo?,size type,base shape,base dimensions,number of studs,has slope?,slope degree,in stock,transparent,Base color
